In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from kaggle.competitions import twosigmanews
from keras.layers import Dense,Activation
from keras.models import Sequential
from keras.optimizers import SGD

In [ ]:
 env =twosigmanews.make_env() 


In [ ]:
(market_df,news_df)=env.get_training_data()
market_df.head()

In [ ]:
market_df.info()

In [ ]:
news_df.info()


In [ ]:
market_assetName=set(market_df.assetName)
market_df=market_df.loc[market_df.universe>0]
news_assetName=news_df.loc[news_df.assetName.isin(market_assetName)]


In [ ]:
market_df['returnsClosePrevMktres10'].corr(market_df['returnsOpenNextMktres10'])

In [ ]:

market_df['o-p']=market_df['close']-market_df['open']



np.cov(market_df['open'],market_df['close'])[1][1]

In [ ]:
#lets check the trend of the asset in the market

news_df.head()



In [ ]:
news_df.groupby(['assetName'])

In [ ]:
news_df.columns

In [ ]:

news_df.drop(['time','sourceTimestamp','sourceId','headline','provider','subjects','audiences','assetName','assetCodes','firstCreated'],axis=1,inplace=True)
news_df.drop(['headlineTag'],axis=1,inplace=True)
news_df.columns

In [ ]:
import xgboost as xgb
news_df.columns

data_news=['sentimentNeutral','sentimentPositive','sentimentNegative','sentimentClass','marketCommentary']
news_df=news_df[data_news]
news_df["returnsOpenNextMktres10"]=market_df['returnsOpenNextMktres10']
news_df.info()

#labels=news_df['returnsOpenNextMktres1']
news_df.info()
#labels=market_df['returnsOpenNextMktres10']

#data_news=xgb.DMatrix(news_df,label=labels,feature_names=news_df.columns)
#n=xgb.train({},data_news)
#xgb.plot_importance(n)

In [ ]:
from sklearn.model_selection import train_test_split
train_indices, val_indices = train_test_split(market_df.index.values,test_size=0.25)

In [ ]:
market_df.columns




In [ ]:
#market_df.drop(['assetName','assetCode','time'],axis=1,inplace=True)
labels=market_df['returnsOpenNextMktres10']
#market_df.drop(['returnsOpenNextMktres10'],axis=1,inplace=True)


In [ ]:
#from above two cells it is claer that news_data is not that usefull for stock_orediction
#retunOpenMrket1 and new data has no compatablity

In [ ]:
new_market_data=['volume','returnsOpenPrevMktres1','close','o-p','returnsOpenPrevRaw1']
market1_df=market_df[new_market_data]
market1_df.columns

In [ ]:
#lets check how relevent this parameters are with returnOpenNextMrket1


In [ ]:
data_news=xgb.DMatrix(market1_df,label=labels,feature_names=market1_df.columns)
market=xgb.train({},data_news)

xgb.plot_importance(market)

In [ ]:
#Now let us Standardize the dataset
#And lets use the data to  train the LSTM network 
labels.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
##labels.drop(axis=0)
##market1_df['returnsOpenNextMktres10']=news_df['returnsOpenNextMktres10']
trans=StandardScaler()
market1_df.dropna()
train=trans.fit_transform(market1_df)
train_data=np.random.rand(len(train))<.7
X_train=train[train_data][:800000]
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))
#X_train=X_train.reshape(80000,5)
                        
X_valid=train[~train_data][:80000]
X_valid=np.reshape(X_valid,(X_valid.shape[0],1,X_valid.shape[1]))
##Xtrain.shape

In [ ]:
from keras.layers import LSTM,Dropout
from keras.losses import mse


dropout=.3
model=Sequential()
model.add(LSTM(128))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('tanh'))
model.add(Dropout(dropout))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(optimizer='RMSprop',loss='mse')


In [ ]:
##market_df['returnsOpenNextMktres10'].dropna()
train_y=np.array(market_df['returnsOpenNextMktres10'])
y_data=np.random.rand(len(train_y))<.7
y_train=train_y[y_data][:800000]

##y_train=y_train.reshape(80000,5)
y_test=train_y[~y_data][:80000]

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

check_point = ModelCheckpoint('model.hdf5',verbose=True, save_best_only=True)
early_stop = EarlyStopping(patience=5,verbose=True)

model.fit(X_train,y_train,epochs=10,verbose=True,validation_split=.2,callbacks=[early_stop,check_point])


In [ ]:
from sklearn.metrics import accuracy_score
model.load_weights('model.hdf5')
confidence_valid = model.predict(X_valid)
print(accuracy_score(confidence_valid.astype('int'),y_test.astype('int')))